In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
path = r"E:\1. Thesis\PDD_MangoLeaf\Dataset"

In [3]:
batch_size = 16
IMG_SHAPE = (128,128,3)
num_of_categories = 8

In [4]:
image_generator = ImageDataGenerator(rescale=1/255,
                                    horizontal_flip=True,
                                    zoom_range=0.2,
                                    validation_split=0.2)

In [5]:
train_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                   directory=path,
                                                   shuffle=True,
                                                   target_size=(128,128),
                                                   subset="training",
                                                   class_mode="categorical")

Found 3200 images belonging to 8 classes.


In [6]:
validation_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                   directory=path,
                                                   shuffle=True,
                                                   target_size=(128,128),
                                                   subset="validation",
                                                   class_mode="categorical")

Found 800 images belonging to 8 classes.


In [7]:
base_model = Xception(weights='imagenet', 
                         include_top=False, 
                         input_shape=IMG_SHAPE)

In [8]:
base_model.trainable=True

In [9]:
inputs = tf.keras.Input(shape=(IMG_SHAPE)) #Define input
x = base_model(inputs,training=False) #pass input to base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
outputs = tf.keras.layers.Dense(num_of_categories, activation='softmax')(x)

model = tf.keras.Model(inputs,outputs)  #Create model


In [10]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 4, 4, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           8,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,967,856 (87.62 MB)

 Trainable params: 22,913,328 (87.41 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [11]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Corrected 'optimizers'
    metrics=['accuracy']
)
